##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimators を使用するブースティング木

<table class="tfo-notebook-buttons" align="left">
  <td> <img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/estimator/boosted_trees.ipynb">TensorFlow.org で表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/estimator/boosted_trees.ipynb">Google Colab で実行</a> </td>
  <td> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/estimator/boosted_trees.ipynb">GitHubでソースを表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/download_logo_32px.png"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/estimator/boosted_trees.ipynb">ノートブックをダウンロード</a> </td>
</table>

> 警告: 新しいコードには Estimators は推奨されません。Estimators は `v1.Session` スタイルのコードを実行しますが、これは正しく記述するのはより難しく、特に TF 2 コードと組み合わせると予期しない動作をする可能性があります。Estimators は、[互換性保証] (https://tensorflow.org/guide/versions) の対象となりますが、セキュリティの脆弱性以外の修正は行われません。詳細については、[移行ガイド](https://tensorflow.org/guide/migrate)を参照してください。

**注意**: 多くの最先端の決定フォレストアルゴリズムの最新の Keras ベースの実装は、[TensorFlow 決定フォレスト](https://tensorflow.org/decision_forests)から利用できます。

このチュートリアルは、`tf.estimator`API で決定木を使用する勾配ブースティングモデルのエンドツーエンドのウォークスルーです。ブースティング木モデルは、回帰と分類の両方のための最も一般的かつ効果的な機械学習アプローチの 1 つです。これは、複数（10 以上、100 以上、あるいは 1000 以上の場合も考えられます）の木モデルからの予測値を結合するアンサンブル手法です。

最小限のハイパーパラメータ調整で優れたパフォーマンスを実現できるため、ブースティング木モデルは多くの機械学習実践者に人気があります。

## Titanic データセットを読み込む

Titanic データセットを使用します。ここでの目標は、性別、年齢、クラスなど与えられた特徴から（やや悪趣味ではありますが）乗船者の生存を予測することです。

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [ ]:
import tensorflow as tf
tf.random.set_seed(123)

データセットはトレーニングセットと評価セットで構成されています。

- `dftrain`と`y_train`は *トレーニングセット*です — モデルが学習に使用するデータです。
- モデルは*評価セット*、`dfeval`、`y_eval`に対してテストされます。

トレーニングには以下の特徴を使用します。

<table>
  <tr>
    <th>特徴名</th>
    <th>説明</th>
  </tr>
  <tr>
    <td>sex</td>
    <td>乗船者の性別</td>
  </tr>
  <tr>
    <td>age</td>
    <td>乗船者の年齢</td>
  </tr>
    <tr>
    <td>n_siblings_spouses</td>
    <td>同乗する兄弟姉妹および配偶者</td>
  </tr>
    <tr>
    <td>parch</td>
    <td>同乗する両親および子供</td>
  </tr>
    <tr>
    <td>fare</td>
    <td>運賃</td>
  </tr>
    <tr>
    <td>class</td>
    <td>船室のクラス</td>
  </tr>
    <tr>
    <td>deck</td>
    <td>搭乗デッキ</td>
  </tr>
    <tr>
    <td>embark_town</td>
    <td>乗船者の乗船地</td>
  </tr>
    <tr>
    <td>alone</td>
    <td>一人旅か否か</td>
  </tr>
</table>

## データを検証する

まず最初に、データの一部をプレビューして、トレーニングセットの要約統計を作成します。

In [ ]:
dftrain.head()

In [ ]:
dftrain.describe()

トレーニングセットと評価セットには、それぞれ 627 個と 264 個の例があります。

In [ ]:
dftrain.shape[0], dfeval.shape[0]

乗船者の大半は 20 代から 30 代です。

In [ ]:
dftrain.age.hist(bins=20)
plt.show()

男性の乗船者数は女性の乗船者数の約 2 倍です。

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')
plt.show()

乗船者の大半は「3 等」の船室クラスを利用していました。

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')
plt.show()

大半の乗船者はサウサンプトンから乗船しています。

In [ ]:
dftrain['embark_town'].value_counts().plot(kind='barh')
plt.show()

女性は男性よりも生存する確率がはるかに高く、これは明らかにモデルの予測特徴です。

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')
plt.show()

## 特徴量カラムを作成して関数を入力する

勾配ブースティング Estimator は数値特徴とカテゴリ特徴の両方を利用します。特徴量カラムは、全ての TensorFlow Estimator と機能し、その目的はモデリングに使用される特徴を定義することにあります。さらに、One-Hot エンコーディング、正規化、バケット化などいくつかの特徴量エンジニアリング機能を提供します。このチュートリアルでは、`CATEGORICAL_COLUMNS`のフィールドはカテゴリカラムから One-Hot エンコーディングされたカラム（[インジケータカラム](https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column)）に変換されます。

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

def one_hot_cat_column(feature_name, vocab):
  return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

特徴量カラムが生成する変換は表示することができます。例えば、`indicator_column`を単一の例で使用した場合の出力は次のようになります。

In [ ]:
example = dict(dftrain.head(1))
class_fc = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('class', ('First', 'Second', 'Third')))
print('Feature value: "{}"'.format(example['class'].iloc[0]))
print('One-hot encoded: ', tf.keras.layers.DenseFeatures([class_fc])(example).numpy())

さらに、特徴量カラムの変換を全てまとめて表示することができます。

In [ ]:
tf.keras.layers.DenseFeatures(feature_columns)(example).numpy()

次に、入力関数を作成する必要があります。これらはトレーニングと推論の両方のためにデータをモデルに読み込む方法を指定します。[ `tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) API の`from_tensor_slices`メソッドを使用して Pandas から直接データを読み取ります。これは小規模でインメモリのデータセットに適しています。大規模のデータセットの場合は、多様なファイル形式（[csv](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)を含む）をサポートする tf.data API を使用すると、メモリに収まりきれないデータセットも処理することができます。

In [ ]:
# Use entire batch since this is such a small dataset.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = dataset.repeat(n_epochs)
    # In memory training doesn't use batching.
    dataset = dataset.batch(NUM_EXAMPLES)
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

## モデルをトレーニングして評価する

以下のステップで行います。

1. 特徴とハイパーパラメータを指定してモデルを初期化する。
2. `train_input_fn`を使用してモデルにトレーニングデータを与え、`train`関数を使用してモデルをトレーニングする。
3. 評価セット（この例では`dfeval` DataFrame）を使用してモデルのパフォーマンスを評価する。予測値が`y_eval`配列のラベルと一致することを確認する。

ブースティング木モデルをトレーニングする前に、まず線形分類器（ロジスティック回帰モデル）をトレーニングしてみましょう。ベンチマークを確立するには、より単純なモデルから始めるのがベストプラクティスです。

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns)

# Train model.
linear_est.train(train_input_fn, max_steps=100)

# Evaluation.
result = linear_est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

次に、ブースティング木モデルをトレーニングしてみましょう。ブースティング木では、回帰（`BoostedTreesRegressor`）と分類（`BoostedTreesClassifier`）をサポートします。目標は、生存か非生存かのクラスを予測することなので、`BoostedTreesClassifier`を使用します。


In [ ]:
# Since data fits into memory, use entire dataset per layer. It will be faster.
# Above one batch is defined as the entire dataset.
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=n_batches)

# The model will stop training once the specified number of trees is built, not
# based on the number of steps.
est.train(train_input_fn, max_steps=100)

# Eval.
result = est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

このトレーニングモデルを使用して、評価セットからある乗船者に予測を立てることができます。TensorFlow モデルは、バッチ、コレクション、または例に対してまとめて予測を立てられるように最適化されています。以前は、`eval_input_fn` は評価セット全体を使って定義されていました。

In [ ]:
pred_dicts = list(est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')
plt.show()

最後に、結果の受信者操作特性（ROC）を見てみましょう。真陽性率と偽陽性率間のトレードオフに関し、より明確な予想を得ることができます。

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)
plt.show()